# 房價預測模型
---
模型02 - 減少模型大小

使用Kaggle資料集預測房屋價格

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
data = pd.read_csv("kc_house_data.csv")
data.shape

pd.options.display.max_columns = 25
data.head()

# data.dtypes

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
data["year"] = pd.to_numeric(data["date"].str.slice(0, 4))
data["month"] = pd.to_numeric(data["date"].str.slice(4, 6))
data["day"] = pd.to_numeric(data["date"].str.slice(6, 8))

data.drop(["id"], axis=1, inplace=True)
data.drop(["date"], axis=1, inplace=True)
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,2014,10,13
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12,9
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,2015,2,25
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2014,12,9
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2015,2,18


In [4]:
data_num = data.shape[0]
indexs = np.random.permutation(data_num)

train_indexs = indexs[:int(data_num*0.6)]
val_indexs = indexs[int(data_num*0.6):int(data_num*0.8)]
test_indexs = indexs[int(data_num*0.8):]

train_data = data.loc[train_indexs]
val_data = data.loc[val_indexs]
test_data = data.loc[test_indexs]

train_data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
21534,599000.0,3,1.75,1650,1180,3.0,0,0,3,8,1650,0,2014,0,98105,47.6636,-122.319,1720,1960,2014,8,11
3713,421000.0,4,2.25,1310,8400,1.0,0,0,4,7,1310,0,1966,0,98034,47.7317,-122.181,1600,8400,2014,11,20
4876,498800.0,2,1.75,1350,4614,1.0,0,0,3,8,1350,0,2008,0,98053,47.7091,-122.015,1680,4775,2014,7,9
14376,546000.0,2,2.25,1530,1324,2.0,0,0,3,8,1280,250,2010,0,98122,47.6032,-122.311,1410,1689,2015,3,9
16539,1000000.0,3,1.00,1880,5450,1.5,0,0,4,8,1880,0,1924,0,98112,47.6453,-122.302,2580,5450,2014,12,15


In [5]:
mean = train_data.mean()
std = train_data.std()
train_data = (train_data - mean) / std

mean = val_data.mean()
std = val_data.std()
val_data = (val_data - mean) / std

# val_data.mean()
# val_data.std()

In [6]:
x_train = np.array(train_data.drop("price", axis=1))
y_train = np.array(train_data["price"])
x_val = np.array(val_data.drop("price", axis=1))
y_val = np.array(val_data["price"])

print(x_val.shape)
print(y_val.shape)

(4323, 21)
(4323,)


In [7]:
model = keras.Sequential(name="model-2")

model.add(layers.Dense(32, activation="relu", input_shape=(21, )))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(1))

print(model.summary())

Model: "model-2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                704       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,793
Trainable params: 1,793
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.compile(keras.optimizers.Adam(0.001), loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.MeanAbsoluteError()])

In [9]:
model_dir = "logs/models/"
if not os.path.exists(model_dir):
    print("new dir!")
    os.makedirs(model_dir)

log_dir = os.path.join("logs", "model-2")
model_cdk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir+"best-model-2.h5", monitor="val_mean_absolute_error", 
                                             save_best_only=True, mode="min")

In [ ]:
history = model.fit(x_train, y_train, batch_size=64, epochs=100,
                    validation_data=(x_val, y_val), callbacks=[model_cdk, model_mckp])

Epoch 1/100
203/203 [==============================] - 2s 9ms/step - loss: 0.3313 - mean_absolute_error: 0.3672 - val_loss: 0.2776 - val_mean_absolute_error: 0.3275
Epoch 2/100
203/203 [==============================] - 1s 6ms/step - loss: 0.2287 - mean_absolute_error: 0.3064 - val_loss: 0.2410 - val_mean_absolute_error: 0.3016
Epoch 3/100
203/203 [==============================] - 1s 6ms/step - loss: 0.2027 - mean_absolute_error: 0.2878 - val_loss: 0.2223 - val_mean_absolute_error: 0.2913
Epoch 4/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1903 - mean_absolute_error: 0.2784 - val_loss: 0.2057 - val_mean_absolute_error: 0.2790
Epoch 5/100
203/203 [==============================] - 1s 5ms/step - loss: 0.1787 - mean_absolute_error: 0.2669 - val_loss: 0.1914 - val_mean_absolute_error: 0.2691
Epoch 6/100
203/203 [==============================] - 1s 6ms/step - loss: 0.1652 - mean_absolute_error: 0.2568 - val_loss: 0.1895 - val_mean_absolute_error: 0.2612
Epoch 7/10

203/203 [==============================] - 1s 5ms/step - loss: 0.0785 - mean_absolute_error: 0.1819 - val_loss: 0.1153 - val_mean_absolute_error: 0.2075
Epoch 51/100
203/203 [==============================] - 1s 5ms/step - loss: 0.0784 - mean_absolute_error: 0.1814 - val_loss: 0.1174 - val_mean_absolute_error: 0.2174
Epoch 52/100
203/203 [==============================] - 1s 5ms/step - loss: 0.0773 - mean_absolute_error: 0.1811 - val_loss: 0.1133 - val_mean_absolute_error: 0.2103
Epoch 53/100
203/203 [==============================] - 1s 5ms/step - loss: 0.0773 - mean_absolute_error: 0.1810 - val_loss: 0.1149 - val_mean_absolute_error: 0.2062
Epoch 54/100
203/203 [==============================] - 1s 5ms/step - loss: 0.0774 - mean_absolute_error: 0.1807 - val_loss: 0.1245 - val_mean_absolute_error: 0.2270
Epoch 55/100
202/203 [============================>.] - ETA: 0s - loss: 0.0778 - mean_absolute_error: 0.1818

In [ ]:
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(loc="upper right")
plt.title('MSE')

In [ ]:
plt.plot(history.history["mean_absolute_error"], label="train")
plt.plot(history.history["val_mean_absolute_error"], label="val")
plt.xlabel("epochs")
plt.ylabel("metrics")
plt.legend(loc="upper right")
plt.title('MAE')

In [ ]:
model.load_weights("logs/models/best-model-2.h5")

y_test = np.array(test_data["price"])
mean = test_data.mean()
std = test_data.std()
test_data = (test_data - mean) / std
x_test = np.array(test_data.drop("price", axis=1))

y_pred = model.predict(x_test)

y_pred = np.reshape(y_pred * std["price"] + mean["price"], y_test.shape)
precentage_error = np.mean(np.abs(y_test - y_pred)) / np.mean(y_test) * 100

print("Model precentage error: {:.2f}%".format(precentage_error))

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs

In [ ]:
# %reload_ext tensorboard